In [0]:
pip install transformers

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 23.9MB/s 
     |████████████████████████████████| 3.7MB 36.3MB/s 
     |████████████████████████████████| 870kB 31.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=998925f805cf263e9ee0859ddfd2df7226cca79d3e22f80f3decead2be7cb6cd
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/Anthony-Alridge/QualityEstimation
%cd 'QualityEstimation'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'QualityEstimation'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 113 (delta 50), reused 88 (delta 27), pack-reused 0
Receiving objects: 100% (113/113), 16.61 MiB | 32.02 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/QualityEstimation


In [0]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from torch.utils import data

In [0]:
f = open('data/en-zh/train.enzh.src') # Open file on read mode
lines_en = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.mt') # Open file on read mode
lines_de = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.scores') # Open file on read mode
scores = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

# print(lines_de)

In [0]:
tokeniser = ppb.XLMTokenizer.from_pretrained('xlm-mlm-17-1280')

datas = []
for i,j in zip(lines_en, lines_de):
  if i is '':break
  datas.append(tokeniser.encode(i,text_pair=j,max_length=80,pad_to_max_length=True))
data_tensor = torch.LongTensor(datas)



In [0]:
model = ppb.XLMForSequenceClassification.from_pretrained('xlm-mlm-17-1280', num_labels = 1)
model.cuda()

XLMForSequenceClassification(
  (transformer): XLMModel(
    (position_embeddings): Embedding(512, 1280)
    (embeddings): Embedding(200000, 1280, padding_idx=2)
    (layer_norm_emb): LayerNorm((1280,), eps=1e-12, elementwise_affine=True)
    (attentions): ModuleList(
      (0): MultiHeadAttention(
        (q_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (k_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (v_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (out_lin): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (1): MultiHeadAttention(
        (q_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (k_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (v_lin): Linear(in_features=1280, out_features=1280, bias=True)
        (out_lin): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (2): MultiHeadAttention(
        (q_lin): Linear(in_features=1280, 

In [0]:
scores_ls = []
for s in scores:
  if s is None or s is '':break
  scores_ls.append(float(s))

labels = torch.FloatTensor(scores_ls)

In [0]:
print(labels.shape)
print(data_tensor.shape)


torch.Size([7000])
torch.Size([7000, 80])


In [0]:
my_dataset = data.TensorDataset(data_tensor,labels) # create your datset
my_dataloader = data.DataLoader(my_dataset,batch_size=16,shuffle=True) 

In [0]:
epochs=4
device=torch.device('cuda')
dtype = torch.long
print_every = 100
optimizer = torch.optim.Adamax(model.parameters())
for e in range(epochs):
        for t, (x, y) in enumerate(my_dataloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.float)
            outputs = model(x, labels=y)
            loss = outputs[0]
            optimizer.zero_grad()

            loss.backward()

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                #check_accuracy(loader_val, model)
                print()

Epoch: 0, Iteration 0, loss = 7.9367

Epoch: 0, Iteration 100, loss = 1.5231

Epoch: 0, Iteration 200, loss = 0.7764

Epoch: 0, Iteration 300, loss = 0.7333

Epoch: 0, Iteration 400, loss = 1.3336

Epoch: 1, Iteration 0, loss = 1.5593

Epoch: 1, Iteration 100, loss = 0.4731

Epoch: 1, Iteration 200, loss = 1.0369

Epoch: 1, Iteration 300, loss = 1.1033

Epoch: 1, Iteration 400, loss = 0.7140

Epoch: 2, Iteration 0, loss = 0.9062

Epoch: 2, Iteration 100, loss = 0.5071

Epoch: 2, Iteration 200, loss = 0.6626

Epoch: 2, Iteration 300, loss = 0.5763

Epoch: 2, Iteration 400, loss = 1.4720

Epoch: 3, Iteration 0, loss = 0.9058

Epoch: 3, Iteration 100, loss = 0.9620

Epoch: 3, Iteration 200, loss = 0.7114

Epoch: 3, Iteration 300, loss = 0.6973

Epoch: 3, Iteration 400, loss = 0.7578



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/NLP Chinese Models/xlm_17/'
model.save_pretrained(path)  # save
tokeniser.save_pretrained(path)  # save

('/content/drive/My Drive/NLP Chinese Models/xlm_17/vocab.json',
 '/content/drive/My Drive/NLP Chinese Models/xlm_17/merges.txt',
 '/content/drive/My Drive/NLP Chinese Models/xlm_17/special_tokens_map.json',
 '/content/drive/My Drive/NLP Chinese Models/xlm_17/added_tokens.json')